# **Many thanks to Adriano Passos for the idea of using RGB images in this competition. In this code, I have minimally modified its code, adding resize 50 by 50 when forming the dataset. To save time, I also used the Adriano Passos dataset for a test sample. In this competition, I used the CNN-LSTM neural network.**
TEST DATASET: https://www.kaggle.com/coldfir3/g2net-cqt-dataset-test-jpgrgb

In [ ]:
fast_sub = True # set this to False to generate the whole dataset
train = True # set this to True to generate the train set
test = True # set this to True to generate the test set

In [ ]:
%%capture
!python -m pip install gwpy
!pip install astropy==4.2.1

In [ ]:
from gwpy.timeseries import TimeSeries
from gwpy.plot import Plot
import numpy as np
from scipy import signal
from PIL import Image

import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from joblib import Parallel, delayed
from pathlib import Path

In [ ]:
from PIL import Image

def sig2rgb(fname, whiten = True, window=0.2, bandpass=False, f_range = (30,400), q_range = (16,32), q_max = 10):
    
    # Load the file 
    data = np.load(fname)
    # Split each chanel and convert to TimeSeries
    data = map(lambda x: TimeSeries(x, sample_rate=2048), data)
    # Whiten the signal and apply a tukey window
    data = map(lambda x: x.whiten(window=("tukey", window)), data)
    # (optional) bandpass filter
    if bandpass:
        data = map(lambda x: x.bandpass(*f_range), data)
    # Q-transform
    data = map(lambda x: x.q_transform(qrange=q_range, frange=f_range, logf=True, whiten=False), data)
    # Convert to RGB image
    img = np.stack(list(data), axis = -1)
    img = np.clip(img, 0, q_max)/q_max * 255
    img = img.astype(np.uint8)
    img = Image.fromarray(img).rotate(90, expand=1)
    img = img.resize((50, 50))
#resized_image.save('resized.png')
    return img

In [ ]:
sig2rgb('../input/g2net-gravitational-wave-detection/train/0/0/0/00000e74ad.npy')


In [ ]:
import glob


def save_img(x, folder_out, **kwargs):
    fname = Path('../input/g2net-gravitational-wave-detection/' + folder_out + '/' + '/'.join([x[0], x[1], x[2], x]) + '.npy')
    file_out = folder_out + '/'+ fname.with_suffix('.jpg').name
    x = sig2rgb(fname, **kwargs)
    x.save(file_out)

In [ ]:
import glob
import shutil
import os

os.makedirs('./kek', exist_ok = True)

src_dir = "../input/g2net-cqt-dataset-test-jpgrgb"
dst_dir = "./kek"
for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
    shutil.copy(jpgfile, dst_dir)


In [ ]:
f = r'./kek'
for file in os.listdir(f):
    f_img = f+"/"+file
    img = Image.open(f_img)
    img = img.resize((50,50))
    img.save(f_img)

from PIL import Image
import os, sys
path = "./kek/"
dirs = os.listdir( path )


def resize():
    for item in dirs:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
            imResize = im.resize((75,75), Image.ANTIALIAS)
            imResize.save(f + '*.jpg', 'JPEG')

resize()           


In [ ]:
train_df = pd.read_csv('../input/g2net-gravitational-wave-detection/training_labels.csv')
if fast_sub: train_ids = train_df['id'][0:20000]
else: train_ids = train_df['id']
if train:
    os.makedirs('train', exist_ok = True)
    o = Parallel(n_jobs=4)(delayed(save_img)(x, 'train') for x in tqdm(train_ids))
    #zip_folder('train')


In [ ]:
path_t = './train/*.jpg'
adres = glob.glob(path_t)
len(adres)

In [ ]:
import cv2
import matplotlib.pyplot as plt

filenames = [img for img in glob.glob("./train/*.jpg")]

filenames.sort() # ADD THIS LINE

images = []
for img in filenames:
    n= cv2.imread(img, cv2.IMREAD_UNCHANGED)
    images.append(n)
    print (img)
plt.imshow(images[0])

In [ ]:
img = cv2.imread('./train/00000e74ad.jpg', cv2.IMREAD_UNCHANGED)
arr = np.array(img)
arr
plt.imshow(img)
sig2rgb('../input/g2net-gravitational-wave-detection/train/0/0/0/00000e74ad.npy')
img.shape


In [ ]:
images = np.array(images)
#images = images.astype('float32')
images = images.reshape(images.shape[0],1, 50, 50, 3)


train_df = train_df[0:20000]
train_df
train_df['target'].hist()
images.shape


In [ ]:
y = train_df['target']

y

In [ ]:
!pip install --upgrade tensorflow
from tensorflow import keras 
from tensorflow.keras import backend as K


from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import ConvLSTM2D
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

In [ ]:
import tensorflow as tf
np.random.seed(500)



#  Обучение нейронной сети 

# Creating a model
model = Sequential()
model.add(TimeDistributed(Conv2D(filters=300, kernel_size=(5,5), padding = "same", activation='relu'), input_shape=(1, 50, 50, 3)))
model.add(Dropout(0.20))
model.add(TimeDistributed(MaxPooling2D(pool_size=(5,5))))
model.add(TimeDistributed(Conv2D(filters=150, kernel_size=(5,5), padding = "same", activation='relu'), input_shape=(1, 50, 50, 3)))
model.add(Dropout(0.20))
model.add(TimeDistributed(MaxPooling2D(pool_size=(5,5))))
model.add(TimeDistributed(Conv2D(filters=75, kernel_size=(5,5), padding = "same", activation='relu'), input_shape=(1, 50, 50, 3)))
model.add(Dropout(0.20))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(150, activation='relu', return_sequences=True))
model.add(Dropout(0.20))
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(Dropout(0.20))
model.add(Dense(1))
model.add(Activation('sigmoid'))


# Compiling model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer='adam', 
              metrics=["accuracy"])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

# Training a model

net_res_1 = model.fit(images, y,
                    epochs=100,
                    verbose = 1, batch_size=200, callbacks=[callback])#, validation_data = (X_test, y_test))



#98

In [ ]:
y_pred = model.predict(images)
y_pred

In [ ]:
test_df = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')
test_df

In [ ]:
filenames = [img for img in glob.glob("./kek/*.jpg")]

filenames.sort() # ADD THIS LINE

images = []
for img in filenames:
    n= cv2.imread(img, cv2.IMREAD_UNCHANGED)
    images.append(n)
    print (img)
plt.imshow(images[0])

In [ ]:
images = np.array(images)
#images = images.astype('float32')
images = images.reshape(images.shape[0],1, 50, 50, 3)

subm_predict = model.predict(images)
subm_predict

import gc
del images

mysor = gc.collect()

# делаем сабмит
test_df = test_df.drop(['target'], axis=1)
test_df['target'] = subm_predict.reshape(len(subm_predict)).tolist()
test_df.to_csv('submission.csv',index = False)
test_df